# Project 3 : 3-NLP

## Problem statement

> #### Create the model to predict where the given post came from between cat and dog post

> - **Create and compare two models**. Any two classifiers at least of your choosing: random forest, logistic regression, KNN, etc.

try to use random search at first

cvec + rf
tfidf + rf

cvec + lr
cvec + ss + lr

cvec + ss + knn : 
tfidf + knn

cvec + nb + gs : try 2grams 3grams

cvec + ss + boost

stack
lv1 = nb + rf + lr
lv2 = lr